In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')

# I just upload all the AUPs in one folder
dir = "/content/drive/My Drive/project/AUP_project/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
docs = []
corpus = ""
for fname in os.listdir(dir):
    doc = open(f'{dir}{fname}', 'r', encoding='unicode_escape').read()
    docs.append(doc)
    corpus += doc

# Extract disallows sentences

In [5]:
import re


# Define disallow keywords
disallow_keywords = ["disallow", "prohibit", "not allowed", "forbidden", "restricted"]

# Extract disallow statements using keyword matching
disallow_statements = []
disallow_corpus = ""
for sentence in re.split('[.!?]\n', corpus):
    for keyword in disallow_keywords:
        if keyword in sentence.lower():
            disallow_statements.append(sentence.strip())
            disallow_corpus += sentence.strip()

# This one sometimes works but it's not working right now and I have no clue

In [12]:
import spacy

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Text to be analyzed
text = corpus
# Process the text with spaCy
doc = nlp(text)

# Initialize a list to store disallowed activities
disallowed_activities = []

# Find the sentence containing the disallowed activities
disallow_keywords = ['prohibited', 'violation', 'suspension', 'termination', 'hacking', 'spamming', 'malware', "disallow", "prohibit", "not allowed", "forbidden", "restricted"]

# isBreak = False
# for keyword in disallow_keywords:
#   for sentence in doc.sents:
#       if keyword in sentence.text.lower():
#           disallowed_sentence = sentence
#           isBreak = True
#           break
#   if isBreak:
#     break

for keyword in disallow_keywords:
  for sentence in doc.sents:
        if keyword in sentence.text.lower():
            disallowed_sentence = sentence
            break

  # Extract disallowed activities from the disallowed sentence
  for token in disallowed_sentence:
      if token.pos_ == "NOUN":
          disallowed_activities.append(token.text)

  # Get the top 10 disallowed activities
  top_10_disallowed_activities = disallowed_activities[:10]

  # Print the top 10 disallowed activities
  print(f"Trying keyword {keyword}: ")
  for i, activity in enumerate(top_10_disallowed_activities, 1):
      print(f"{i}. {activity}")

Trying keyword prohibited: 
1. USES
2. site
3. purposes
Trying keyword violation: 
1. USES
2. site
3. purposes
4. Violations
5. Policy
Trying keyword suspension: 
1. USES
2. site
3. purposes
4. Violations
5. Policy
6. .
7. TERMINATION
8. discretion
9. breach
10. use
Trying keyword termination: 
1. USES
2. site
3. purposes
4. Violations
5. Policy
6. .
7. TERMINATION
8. discretion
9. breach
10. use
Trying keyword hacking: 
1. USES
2. site
3. purposes
4. Violations
5. Policy
6. .
7. TERMINATION
8. discretion
9. breach
10. use
Trying keyword spamming: 
1. USES
2. site
3. purposes
4. Violations
5. Policy
6. .
7. TERMINATION
8. discretion
9. breach
10. use
Trying keyword malware: 
1. USES
2. site
3. purposes
4. Violations
5. Policy
6. .
7. TERMINATION
8. discretion
9. breach
10. use
Trying keyword disallow: 
1. USES
2. site
3. purposes
4. Violations
5. Policy
6. .
7. TERMINATION
8. discretion
9. breach
10. use
Trying keyword prohibit: 
1. USES
2. site
3. purposes
4. Violations
5. Policy
6. .

# This one doesn't make sense

In [7]:
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Text to be analyzed
text = disallow_corpus

# Initialize the RAKE object
r = Rake()

# Extract keywords
r.extract_keywords_from_text(text)

# Get the top 10 keywords
top_10_keywords = r.get_ranked_phrases()[:10]

# Print the top 10 keywords
for i, keyword in enumerate(top_10_keywords, 1):
    print(f"{i}. {keyword}")


1. callbacknavigation navigation navigation navigation exams exams supercoaching live classes header_free test series skill academy select courses practice pass pass pro gk
2. streaming media serversproxy serversremote adminstration serversdns serversdhcp serversx serverstelnet serversssh serversprint serverstftp serversmessaging serversmud serversicq serverschat serversirc serversdatabase serversfile
3. streaming media serversproxy serversremote adminstration serversdns serversdhcp serversx serverstelnet serversssh serversprint serverstftp serversmessaging serversmud serversicq serverschat serversirc serversdatabase serversfile
4. streaming media serversproxy serversremote adminstration serversdns serversdhcp serversx serverstelnet serversssh serversprint serverstftp serversmessaging serversmud serversicq serverschat serversirc serversdatabase serversfile
5. exams english hindi get started get started english hindi get started get pass pro renew pass pro get pass pro upgrade
6. appsas

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
